In [32]:
import os
os.chdir('/Users/mumtaz/Documents/projects/zavmo/zavmo-api')

In [33]:
import pandas as pd
from IPython.display import Markdown
import json

In [165]:
ofqual_SSAs = json.load(open(f"{os.getcwd()}/zavmo/classification/CrossValidation/data/ofqual_SSAs.json"))

In [166]:
## Latest ofqual data
existing_ofqual_filepath = "/Users/mumtaz/Documents/projects/zavmo/ofqual_unit_details.csv"

d1 = pd.read_csv(existing_ofqual_filepath)
print(d1.shape)
d1.head(3)

(26579, 6)


,qualification_id,overview,id,title,description,learning_outcomes
0,601/0405/3,This is an entry-level qualification that enab...,A/001/0001,Constructing halving joints,Learners will learn how to construct halving j...,"[{""description"": ""Understand the principles of..."
1,601/0405/3,This is an entry-level qualification that enab...,A/001/0002,Constructing housing joints,Learners will learn how to construct housing j...,"[{""description"": ""Understand the principles of..."
2,601/0405/3,This is an entry-level qualification that enab...,A/001/0003,Painting techniques,Learners will acquire essential painting techn...,"[{""description"": ""Demonstrate key painting tec..."


In [ ]:
new_ofqual_filepath = "/Users/mumtaz/Documents/projects/zavmo/ofqual_unit_details (1).csv"
d2 = pd.read_csv(new_ofqual_filepath)
print(d2.shape)
d2.head(3)

In [163]:
ofqual = pd.concat([d1, d2], ignore_index=True)
ofqual.shape

(27736, 6)

In [ ]:
## After appending the data save it
ofqual.to_csv("/Users/mumtaz/Documents/projects/zavmo/ofqual_unit_details.csv", index=False)

## Mapping SSA and Sub SSA

In [168]:
ofqual_SSAs_data = pd.read_csv("/Users/mumtaz/Documents/projects/zavmo/zavmo-api/docs/nos-ofqual/ofqual_details_20250127.csv.zip")
ofqual_SSAs_data.columns

Index(['qualification_type', 'qualification_level', 'qualification_number',
       'assessment_methods', 'sector_subject_area', 'awarding_organisation',
       'grading_type', 'total_credits', 'funding_in_england', 'status',
       'national_availability', 'regulation_start_date',
       'operational_start_date', 'operational_end_date',
       'final_certification_date', 'european_qualification_level', 'ofqual_id',
       'guided_learning_hours', 'total_qualification_time', 'specification',
       'specialisms', 'optional_routes', 'grading_scale',
       'end-point_assessment_standard'],
      dtype='object')

In [169]:
ofqual.rename(columns={'qualification_id': 'ofqual_id'}, inplace=True)

In [149]:
import numpy as np
len(np.unique(ofqual['ofqual_id']))

3352

In [150]:
len(ofqual_SSAs_data.ofqual_id.unique())

24545

In [151]:
len(ofqual_SSAs_data['sector_subject_area'].unique())

47

In [ ]:
ofqual_before_merge = ofqual.copy()  # Store the original dataframe before merge

In [171]:
ofqual = ofqual.merge(ofqual_SSAs_data[['ofqual_id', 'sector_subject_area']], 
    on='ofqual_id',
    how='inner')


In [176]:
missed_records = ofqual_before_merge[~ofqual_before_merge['ofqual_id'].isin(ofqual['ofqual_id'])]
missed_records.reset_index(drop=True, inplace=True)

In [177]:
missed_records.reset_index(drop=True, inplace=True)

In [180]:
missed_records.to_csv("/Users/mumtaz/Documents/projects/zavmo/zavmo-api/zavmo/classification/CrossValidation/data/missed_ofqual_records.csv", index=False)

In [152]:
ofqual.rename(columns={'sector_subject_area': 'sub_SSA'}, inplace=True)

## SSA mapping to Sub SSA

category_mapping = {
    sub_category: parent_category
        for category_dict in ofqual_SSAs 
        for parent_category, sub_categories in category_dict.items()
        for sub_category in sub_categories
}

def assign_parent_category(sector):
    matches = [parent for sub_cat, parent in category_mapping.items() if sub_cat in sector]
    return matches[0] if matches else None

ofqual['SSA'] = (ofqual['sub_SSA'].map(assign_parent_category))


In [153]:
ofqual

,ofqual_id,overview,id,title,description,learning_outcomes,sub_SSA,SSA
0,601/0405/3,This is an entry-level qualification that enab...,A/001/0001,Constructing halving joints,Learners will learn how to construct halving j...,"[{""description"": ""Understand the principles of...",Building and construction,"Construction, Planning and the Built Environment"
1,601/0405/3,This is an entry-level qualification that enab...,A/001/0002,Constructing housing joints,Learners will learn how to construct housing j...,"[{""description"": ""Understand the principles of...",Building and construction,"Construction, Planning and the Built Environment"
2,601/0405/3,This is an entry-level qualification that enab...,A/001/0003,Painting techniques,Learners will acquire essential painting techn...,"[{""description"": ""Demonstrate key painting tec...",Building and construction,"Construction, Planning and the Built Environment"
3,601/0405/3,This is an entry-level qualification that enab...,A/001/0004,Tiling a plain wall,Learners will learn how to tile a plain wall e...,"[{""description"": ""Understand and apply correct...",Building and construction,"Construction, Planning and the Built Environment"
4,601/0405/3,This is an entry-level qualification that enab...,A/001/0005,Constructing half brick walling,Learners will learn the skills required to con...,"[{""description"": ""Demonstrate the ability to c...",Building and construction,"Construction, Planning and the Built Environment"
...,...,...,...,...,...,...,...,...
21982,603/1035/2,The qualification aims to enable candidates to...,M/615/4703,"Strategically Manage Quarrying Safety, Health ...",Focuses on strategic management of safety in q...,"[{""description"": ""Develop strategic plans for ...",Manufacturing technologies,Engineering and Manufacturing Technologies
21983,603/1035/2,The qualification aims to enable candidates to...,F/615/2454,Knowledge of Drilling and Blasting for Quarries,Provides essential knowledge about blasting te...,"[{""description"": ""Understand drilling and blas...",Manufacturing technologies,Engineering and Manufacturing Technologies
21984,603/1035/2,The qualification aims to enable candidates to...,D/615/2462,Manage Quarry Drilling and Blasting,Covers the management aspects of drilling and ...,"[{""description"": ""Effectively manage drilling ...",Manufacturing technologies,Engineering and Manufacturing Technologies
21985,603/4162/2,This qualification unit focuses on developing ...,MUS/372/1,Mastro Sain mewn Stiwdio – Mewnol,This unit covers the essential skills required...,"[{""description"": ""Demonstrate skills in master...",Building and construction,"Construction, Planning and the Built Environment"


In [154]:
len(ofqual['ofqual_id'].unique())

2514

In [155]:
ofqual.to_csv("/Users/mumtaz/Documents/projects/zavmo/zavmo-api/docs/nos-ofqual/ofqual.csv", index=False)

In [156]:
ofqual.shape

(21987, 8)